In [1]:
args = {
    "dataset_folder": "./train_10M",
    "epochs": 5
}

In [2]:
import os
from datasets import load_dataset
import datasets
import torch

In [3]:
!mkdir curricula

mkdir: cannot create directory ‘curricula’: File exists


### Curriculum 2023 dataset

The exact data from Thoma et al 2023. Note that the model does not see all examples in each epoch.

In [4]:
from datasets import load_dataset
import os
os.environ['HF_HOME'] = '/data/loriss21dm/cache'
# dev = load_dataset("babylm-anon/dev-data")

In [5]:
LAST_GEN_DIR = "last_gen"
EPOCHS_PER_STAGE = 1
datasets_stages = []
orders_stages = []

torch.manual_seed(0)
for folder in (sorted(os.listdir(LAST_GEN_DIR))):
    d = load_dataset("text", data_dir =os.path.join(LAST_GEN_DIR, folder), cache_dir='/data/loriss21dm/cache')["train"] 
    d = d.shuffle() # we shuffle within the stage
    d = datasets.Dataset.from_dict(d[0:len(d)//10]) # TODO
    datasets_stages.append(d)
    offset = orders_stages[-1][-1]+1 if len(orders_stages) else 0
    for _ in range(0,EPOCHS_PER_STAGE):
        orders_stages.append(torch.arange(offset,len(d)+offset))
    


dataset = datasets.concatenate_datasets(datasets_stages)
dataset.save_to_disk("./curricula/datasets/curriculum_100M_2023")
#assert torch.equal(torch.cat([o.flatten() for o in orders_stages]),torch.arange(0,len(dataset)))

torch.save(orders_stages, "./curricula/curriculum_100M_2023")
len(orders_stages)

Saving the dataset (0/1 shards):   0%|          | 0/1058753 [00:00<?, ? examples/s]

4

In [6]:
sum((len(o) for o in orders_stages)) / 10587551 


0.09999980165384799

In [7]:
dataset

Dataset({
    features: ['text'],
    num_rows: 1058753
})

In [8]:
torch.tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]).repeat(1,10,1),

(tensor([[[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
           17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
           34, 35],
          [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
           17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
           34, 35],
          [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
           17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
           34, 35],
          [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
           17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
           34, 35],
          [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
           17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
           34, 35],
          [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
           17, 18, 19, 20, 21, 

### Curriculum with 10M 2024 dataset

For training with the strategy in Thoma et al 2023. Note that the model does not see all examples in each epoch.

In [9]:
args["dataset_folder"] = "./train_10M"
curriculum = {
    "C1": ["childes.train", "open_subtitles.train"],
    "C2": ["switchboard.train", "bnc_spoken.train"],
    "C3": ["gutenberg.train", "simple_wiki.train"]
}

In [10]:
datasets_stages = []
orders_stages = []

torch.manual_seed(0)
for stage, files in curriculum.items():
    d = datasets.concatenate_datasets(
            [
                load_dataset("text", data_files =os.path.join(args["dataset_folder"], file))["train"] 
                for file in files
            ]
        )
    d = d.shuffle() # we shuffle with the stage
    datasets_stages.append(d)
    offset = orders_stages[-1][-1]+1 if len(orders_stages) else 0
    orders_stages.append(torch.arange(offset,len(d)+offset))
    


dataset = datasets.concatenate_datasets(datasets_stages)
dataset.save_to_disk("./curricula/datasets/curriculum_10M_2024")
assert torch.equal(torch.cat([o.flatten() for o in orders_stages]),torch.arange(0,len(dataset)))

torch.save(orders_stages, "./curricula/curriculum_10M_2024")
orders_stages

Saving the dataset (0/1 shards):   0%|          | 0/1179014 [00:00<?, ? examples/s]

[tensor([     0,      1,      2,  ..., 939997, 939998, 939999]),
 tensor([ 940000,  940001,  940002,  ..., 1047997, 1047998, 1047999]),
 tensor([1048000, 1048001, 1048002,  ..., 1179011, 1179012, 1179013])]

### Random Shuffle

This mimicks the default behaviour of Huggingface's `Trainer`: Randomly shuffle dataset after each epoch.
All data is shown at each epoch.

In [11]:
from datasets import load_dataset
dataset = datasets.load_from_disk("./curricula/datasets/curriculum_10M_2024") # reuse one created above

In [12]:

dataset.set_epoch("ha")

AttributeError: 'Dataset' object has no attribute 'set_epoch'

In [131]:
assert torch.equal(torch.sort(torch.randperm(len(dataset))).values, torch.arange(0,len(dataset)))

In [133]:
args["epochs"] = 3
import torch 
torch.manual_seed(0)
random_order = torch.stack([torch.randperm(len(dataset)) for _ in range(0,args["epochs"])])
torch.save(random_order, "./curricula/random_curriculum")

In [138]:
random_order

torch.Size([5, 1179014])

### Curriculum Convolution with lognorm 

## Test random

In [13]:
from datasets import load_dataset
dataset = load_dataset("text", data_dir ="/data/loriss21dm/babylm/train_test", cache_dir='/data/loriss21dm/cache')["train"] 
import torch 
torch.manual_seed(0)
random_order = torch.stack([torch.randperm(len(dataset)) for _ in range(0,args["epochs"])])
torch.save(random_order, "./curricula/test_random_curriculum")

FileNotFoundError: Directory /data/loriss21dm/babylm/train_test is neither a `Dataset` directory nor a `DatasetDict` directory.